In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import lightgbm as lgb
import xgboost as xgb
import scipy
from scipy import stats
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
import seaborn as sns
import dill
import datetime as dt
import pickle
import random
import math
import time
import os
import re
import colorsys
import collections

from sklearn.model_selection import train_test_split
from feature_engine.imputation import CategoricalImputer
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import ArbitraryNumberImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from feature_engine.encoding import OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder as skOrdinalEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.utils import column_or_1d
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from feature_engine.transformation import LogCpTransformer
from feature_engine.outliers import OutlierTrimmer
from scipy.stats import norm, zscore
import matplotlib.lines as mlines
from sklearn.impute import KNNImputer

from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.stats.descriptivestats import Description
from scipy.stats import bootstrap
from sklearn.model_selection import cross_validate
from statsmodels.graphics.regressionplots import plot_partregress_grid
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from _classes import OverviewTransformer
from _classes import LabelTransformer
from _classes import FeaturesCreator
from _classes import FeaturesLogger
from _classes import Scaler
from _classes import Encoder
from _classes import HousePricePredictor
from _classes import Cleaner
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from statsmodels.stats.outliers_influence import reset_ramsey
from sklearn.model_selection import cross_validate
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import KFold

In [2]:
def saturate_color(color_rgb, saturation=0.75):
    color_hls = colorsys.rgb_to_hls(
        color_rgb[0], color_rgb[1], color_rgb[2])
    color_hls_saturated = (
        color_hls[0], color_hls[1], saturation*color_hls[2])
    color_rgb_saturated = colorsys.hls_to_rgb(
        color_hls_saturated[0], color_hls_saturated[1], color_hls_saturated[2])
    return color_rgb_saturated


def saturate_palette(palette, saturation=0.75):
    palette_saturated = [saturate_color(i, saturation=saturation) for i in palette]
    return palette_saturated


def alpha_color(color, alpha=0.75):
    new_color = tuple (x + (1 - x) * (1 - alpha) for x in color)
    return new_color


def alpha_palette(palette, alpha=0.75):
    palette_alphed = [alpha_color(i, alpha=alpha) for i in palette]
    return palette_alphed

In [ ]:
theme = 'light'
# theme = 'dark'

scatter = {
    'lw': 0.15
}
point = {
    'linestyle': 'none',
    'markersize': 3,
    'err_kws': {'lw': 1.2},
    'capsize': 0.03
}
hist = {
    'alpha': 1
}
title_inline = {
    'pad': 27
}
legend = {
    'loc': 'center left', # right and upper (6)
    'bbox_to_anchor': (1,1),
    'frameon': False
}
legend_mid = {
    'loc': 'upper left', # right and mid (2)
    'bbox_to_anchor': (1,1),
    'frameon': False
}
legend_inline = {
    'loc': 'lower left',
    'bbox_to_anchor': (0,1),
    'frameon': False
}
custom_params = {
    'figure.figsize': (8, 2.5), # ширина и высота в дюймах
    'figure.titlesize': 12,
    'figure.titleweight': 'medium',
    'figure.labelsize': 9,
    'figure.labelweight': 'medium',
    'axes.axisbelow': True,
    'axes.spines.bottom': True, # ось X
    'axes.spines.left': True, # ось Y
    'axes.spines.right': False, # не отображать правую рамку 
    'axes.spines.top': False, # не отображать верхнюю рамку
    'axes.linewidth': 1, # 1.2
    'axes.titlelocation': 'left', # расположение заголовка
    'axes.titlepad': 15.0, # расстояние между заголовком и графиком
    'axes.titlesize': 10.0, # заголовок
    'axes.titleweight': 'medium', # заголовок
    'axes.labelpad': 8.0, # расстояние между подписью оси и осью
    'axes.labelsize': 9.0, # оси
    'axes.labelweight': 'bold', # подписи осей
    'xaxis.labellocation': 'left', # положение подписи
    'yaxis.labellocation': 'top', # положение подписи
    'axes.grid': True,
    'axes.grid.axis': 'both', # какую разметку отображать
    'grid.linestyle': ':',
    'grid.linewidth': 0.5,
    'xtick.bottom': True,
    'xtick.direction': 'in',
    'xtick.major.bottom': True,
    'xtick.major.pad': 3.5,
    'xtick.major.size': 3.0,
    'xtick.major.top': True,
    'xtick.major.width': 1,
    'xtick.labelsize': 8.0, # деления оси X
    'ytick.left': True,
    'ytick.direction': 'in',
    'ytick.major.left': True,
    'ytick.major.pad': 3.5,
    'ytick.major.size': 3.0,
    'ytick.major.left': True,
    'ytick.major.width': 1,
    'ytick.labelsize': 8.0, # деления оси Y
    'lines.linewidth': 1.2,
    'lines.markersize': 3,
    'legend.title_fontsize': 8.0,
    'legend.fontsize': 9.0,
    'legend.framealpha': 0.75,
    'font.family': 'Ubuntu'
}
custom_params_light = {
    'figure.facecolor': '#FEFEFE', # FDFDFD
    'figure.edgecolor': '#FEFEFE',
    'axes.facecolor': '#FEFEFE',
    'axes.titlecolor': '#303030',
    'axes.labelcolor': '#505050',
    'axes.edgecolor': '#E6E6E6', # цвет осей
    'grid.color': '#D9D9D9',  
    'xtick.color': '#E6E6E6', # 0.90
    'xtick.labelcolor': '#9F9F9F',
    'ytick.color': '#E6E6E6', # 0.90
    'ytick.labelcolor': '#9F9F9F',
    'legend.labelcolor': '#4B4B4B',
    'legend.edgecolor': '#A6A6A6',
    'legend.facecolor': '#FEFEFE',
    'text.color': '#4B4B4B',
    'patch.force_edgecolor': True, # отображать границу столбцов
    'patch.edgecolor': '#FEFEFE', # цвет границы стобцов
    'scatter.edgecolors': '#FEFEFE'
}
custom_params_dark = {
    'figure.facecolor': '#303030',
    'axes.facecolor': '#303030',
    'axes.edgecolor': '#555555',
    'axes.titlecolor': '#AAAAAA',
    'axes.labelcolor': '#AAAAAA',
    'grid.color': '#CCCCCC', # CCCCCC
    'grid.linewidth': 0.1,
    'xtick.color': '#858585',
    'ytick.color': '#858585',
    'lines.markersize': 3.5,
    'legend.edgecolor': '#555555',
    'legend.facecolor': '#303030',
    'legend.labelcolor': '#858585',
    'legend.framealpha': 0.5,
    'text.color': '#AAAAAA',
    'patch.edgecolor': '#303030'
}

if theme == 'light':

    palette_bright = sns.color_palette([
    #    0 : 0.3    1 red      2 blue     3 green    4 yellow
        '#4B4B4B', '#AF4035', '#4682B4', '#2C8B6D', '#FFA500',
    #    5 brown    6 khaki    7 purple   8 pink     9 orange
        '#6C4839', '#C3B091', '#534D89', '#D07190', '#DE6535',
    #   -10 dark   -9 blue    -8 blue    -7 blue    -6 ligt
        '#304E68', '#3D6384', '#4D7DA8', '#6D9BC3', '#A7CBE8',
    #   -5 : 0.2   -4 : 0.5   -3 : 0.65  -2 : 0.75  -1 : 0.85
        '#353535', '#7F7F7F', '#A5A5A5', '#BFBFBF', '#D9D9D9'
    ])
    palette = saturate_palette(palette_bright, saturation=0.85)
    palette = alpha_palette(palette, alpha=0.90)
    # add palette to custom parameters dictionary for light theme
    custom_params_light.update({'axes.prop_cycle': plt.cycler(color=palette)})

    corr_matrix = sns.color_palette(
        'blend:#255986,#FFFFFF,#933737', as_cmap=True)
    report = sns.color_palette(
        'blend:#E6E9EC,#FFFFFF,#9E2A2B', as_cmap=True)
    gradient = sns.color_palette(
        'blend:#FFFFFF,#3D6384', as_cmap=True)
    pairplot = sns.color_palette(['#8DB4D7', '#AF4035'])

    # add custom parameters of light theme to parameters dictionary
    custom_params.update(custom_params_light)
    # update rcParams 
    plt.rcParams.update(custom_params)
    # parameters for axis_rstyle function
    custom_axis_linewidth = 0.75
    custom_axis_color = '#BBBBBB'

if theme == 'dark':
    
    palette = sns.color_palette([
        '#808080', '#804740', '#748BAA', '#55765B', '#A57C54',
        '#9A6484', '#5B3256', '#6C4839', '#5C8DA0', '#847637',
        '#304E68', '#3D6384', '#4D7DA8', '#6D9BC3', '#A7CBE8',
        '#505050', '#606060', '#707070', '#808080', '#AAAAAA',
    ])
    # add palette to custom parameters dictionary for dark theme
    custom_params_dark.update({'axes.prop_cycle': plt.cycler(color=palette)})
    
    corr_matrix = sns.color_palette(
        'blend:#285780,#909090,#8D3131', as_cmap=True)
    report = sns.color_palette(
        'blend:#E6E9EC,#CCCCCC,#9E2A2B', as_cmap=True)
    gradient = sns.color_palette(
        'blend:#FFFFFF,#3D6384', as_cmap=True)
    pairplot = sns.color_palette(['#8DB4D7', '#AF4035'])

    # add custom parameters of dark theme to parameters dictionary
    custom_params.update(custom_params_dark)
    # update rcParams 
    plt.rcParams.update(custom_params)

%matplotlib inline

seed = 42

In [4]:
sns.color_palette(palette[:5])

[(0.32588235294117646, 0.32588235294117646, 0.32588235294117646),
 (0.6853529411764706, 0.3523529411764706, 0.3193529411764706),
 (0.3761764705882353, 0.5561764705882352, 0.7061764705882353),
 (0.2804411764705882, 0.5654411764705882, 0.4754411764705882),
 (0.9325, 0.6625, 0.16749999999999993)]

In [5]:
sns.color_palette(palette[5:10])

[(0.4676764705882353, 0.35967647058823526, 0.3146764705882353),
 (0.7749999999999999, 0.718, 0.625),
 (0.4056470588235293, 0.38764705882352934, 0.5676470588235294),
 (0.808970588235294, 0.5239705882352942, 0.6169705882352938),
 (0.8387941176470588, 0.47579411764705876, 0.3317941176470588)]

In [6]:
sns.color_palette(palette[10:15])

[(0.28423529411764703, 0.37423529411764694, 0.452235294117647),
 (0.3340882352941177, 0.44808823529411757, 0.5470882352941177),
 (0.39585294117647046, 0.5398529411764704, 0.6688529411764705),
 (0.507470588235294, 0.645470588235294, 0.7654705882352942),
 (0.7066176470588236, 0.8146176470588233, 0.9016176470588235)]

In [7]:
sns.color_palette(palette[15:])

[(0.26941176470588235, 0.26941176470588235, 0.26941176470588235),
 (0.38235294117647056, 0.38235294117647056, 0.38235294117647056),
 (0.49529411764705883, 0.49529411764705883, 0.49529411764705883),
 (0.608235294117647, 0.608235294117647, 0.608235294117647),
 (0.7599999999999999, 0.7599999999999999, 0.7599999999999999)]

In [8]:
plt.rcParams.keys()

KeysView(RcParams({'_internal.classic_mode': False,
          'agg.path.chunksize': 0,
          'animation.bitrate': -1,
          'animation.codec': 'h264',
          'animation.convert_args': ['-layers', 'OptimizePlus'],
          'animation.convert_path': 'convert',
          'animation.embed_limit': 20.0,
          'animation.ffmpeg_args': [],
          'animation.ffmpeg_path': 'ffmpeg',
          'animation.frame_format': 'png',
          'animation.html': 'none',
          'animation.writer': 'ffmpeg',
          'axes.autolimit_mode': 'data',
          'axes.axisbelow': True,
          'axes.edgecolor': '0.90',
          'axes.facecolor': '#FEFEFE',
          'axes.formatter.limits': [-5, 6],
          'axes.formatter.min_exponent': 0,
          'axes.formatter.offset_threshold': 4,
          'axes.formatter.use_locale': False,
          'axes.formatter.use_mathtext': False,
          'axes.formatter.useoffset': True,
          'axes.grid': True,
          'axes.grid.axis': 'bot